In [39]:
import pandas as pd
import os
from sqlalchemy import create_engine
import numpy as np
import re

def remove_links_from_excel(excel_path, output_path):
    from openpyxl import load_workbook

    # Load the workbook
    workbook = load_workbook(excel_path)
    sheet = workbook.active

    # Iterate through cells in the sheet
    for row in sheet.iter_rows():
        for cell in row:
            if cell.hyperlink:
                # Store the cell value
                cell_value = cell.value
                # Remove the hyperlink
                cell.hyperlink = None
                # Restore the cell value
                cell.value = cell_value

    # Save the modified workbook
    workbook.save(output_path)

def create_db_from_excel(excel_path, db_path):
    """
    Reads an Excel file and creates a SQLite database from its sheets.
    Each sheet is converted into a separate table with proper header handling.
    """
    if os.path.exists(db_path):
        os.remove(db_path)  # Remove existing DB to start fresh
        
    xls = pd.ExcelFile(excel_path)
    engine = create_engine(f"sqlite:///{db_path}")
    
    # Process each sheet in the Excel file
    for sheet_idx, sheet_name in enumerate(xls.sheet_names):
        try:
            # First, read the Excel sheet without setting headers
            if sheet_idx == 0:  # First table (summary table)
                # For the first table, use the first row as header
                df = pd.read_excel(xls, sheet_name=sheet_name)
                print(f"Processing summary table '{sheet_name}' with standard headers")

            elif sheet_idx == 2:
                # remove the top two rows and set the third row as header
                df = pd.read_excel(xls, sheet_name=sheet_name, header=2)

                # convert the first two columns to string
                df.iloc[:, 0:2] = df.iloc[:, 0:2].astype(str)

                hyperlink_pattern = r'=HYPERLINK\s*\(\s*"[^"]*"\s*,\s*"([^"]*)"\s*\)'
            
                # Get the first two column names
                first_two_cols = list(df.columns)[:2] if len(df.columns) >= 2 else list(df.columns)
                
                for col in first_two_cols:
                    # Apply the extraction to each cell in the column
                    df[col] = df[col].apply(lambda x: 
                                        re.search(hyperlink_pattern, str(x)).group(1) 
                                        if isinstance(x, str) and re.search(hyperlink_pattern, str(x)) 
                                        else x)
                print(f"Processing table '{sheet_name}' with standard headers")
                # display(df.head())  # Display the first few rows of the DataFrame

                # parse the first two columns as strings, split by comma

            elif sheet_idx == 5:  # Second table (summary table)
                # For the second table, use the first row as header
                df = pd.read_excel(xls, sheet_name=sheet_name)

                print(f"Processing summary table '{sheet_name}' with standard headers")

            elif sheet_idx == 8:  # Second table (detailed table)
                # For other tables, we need to handle the multi-row headers
                # Read the sheet with header=None to get all rows
                df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
                
                if len(df) >= 3:  # Make sure we have enough rows
                    # Get the header rows
                    top_row = df.iloc[0].replace({np.nan: None, 'NaN': None})  # Row with potential suffixes
                    third_row = df.iloc[2].replace({np.nan: None, 'NaN': None})  # Row with base column names
                    
                    # Create combined headers
                    combined_headers = []
                    current_suffix = None
                    
                    for i in range(len(top_row)):
                        # Update suffix if we encounter a non-None value in the top row
                        if top_row[i] is not None:
                            current_suffix = str(top_row[i])
                        
                        # Get base column name from third row
                        base_name = str(third_row[i]) if third_row[i] is not None else f"col_{i}"
                        
                        # Create combined column name
                        if current_suffix:
                            combined_headers.append(f"{base_name}_{current_suffix}")
                        else:
                            combined_headers.append(base_name)
                    
                    # Set combined headers and drop the first three rows
                    df.columns = combined_headers
                    df = df.iloc[3:]
                    print(f"Processed table '{sheet_name}' with combined headers")
                    
                else:
                    print(f"Sheet '{sheet_name}' doesn't have enough rows for header processing")
            else:  # For other tables, use the first row as header
                continue
            
            # Reset index to ensure proper SQLite import
            df = df.reset_index(drop=True)
            
            # Clean column names for SQL compatibility
            df.columns = [str(col).replace(' ', '_').replace('(', '').replace(')', '').replace('.', '_')
                         .replace('-', '_').replace('/', '_').replace('\\', '_') for col in df.columns]
            
            # Sanitize table name
            table_name = ''.join(e for e in sheet_name if e.isalnum() or e == '_')
            if not table_name:  # if sheet name was all special chars
                table_name = f"table_{sheet_idx}"

            display(df)  # Display the first few rows of the DataFrame

            df.to_sql(table_name, engine, index=False, if_exists='replace')
            print(f"Sheet '{sheet_name}' imported as table '{table_name}' with {len(df)} rows.")
            
        except Exception as e:
            print(f"Could not import sheet '{sheet_name}': {e}")
    
    return engine

In [40]:
path = 'Walmart_Tacos_Report_Flying_Star_2025-01-01_to_2025-05-15_May_16_2025_0442.xlsx'
db_path = 'walmart_tacos.db'
engine = create_db_from_excel(path, db_path)

Processing summary table 'Summary' with standard headers


,Duration_135_days,01_01_2025___05_15_2025Current_Data,08_19_2024___12_31_2024Previous_Data,01_01_2024___05_15_2024Last_Year_Data
0,TACOS,0.0719,0.0908,0.0583
1,ROAS,3.2800,3.7100,4.4100
2,ACOS,0.3100,0.2700,0.2300
3,Ad Spend,8339.6400,18141.2600,8227.1100
4,Ad Sales,27342.3500,67366.1000,36261.0400
5,Ad SKU Sales,24940.3600,60828.1000,15393.0400
6,Organic Sales,88589.0300,132381.5000,104797.6700
7,Total Net Sales,115931.3800,199747.6000,141058.7100
8,Total Net Unit Sold,6345.0000,9796.0000,7291.0000
9,Total Published SKU,2379.0000,NaN,NaN


Sheet 'Summary' imported as table 'Summary' with 12 rows.
Processing table 'By Item IDs' with standard headers


,Item_ID,Product_Name,SKU,Category,Product_Type,Brand,Average_Rating,Review_Count,Tags,Total_Net_Sales,...,CPC,Walmart_Item_Page_Views,EBC_Page_Views,Organic_Views,Organic_Sales,Organic_Units_Sold,Organic_Conversion,Share_of_Organic,Days_of_Inventory,Status
0,775282043,"Tirrinia Insulated Lunch Bag for Women, Men, a...",1TRLB701CH-N,"Home Decor, Kitchen, & Other",Reusable Lunch Bags & Boxes,Tirrinia,4.6,256,"Q2-Target,Brand Camp VD,Lunch Bag,WFS,Lunch Ba...",9081.34,...,0.00,23727,3392,20335.0,9081.34,785,0.0386,1.0000,34,Published
1,809495686,Kritter Planet Waterproof Dog Blanket for Couc...,1KPBW686NY,Animal Accessories,Pet Blankets,Kritter Planet,0.0,0,Batch 1 - TR,1778.93,...,0.38,703,190,219.0,1186.21,58,0.2648,0.6668,163,Published
2,849129071,Kritter Planet Waterproof Dog Blanket for Couc...,1KPBW686GY,Animal Accessories,Pet Blankets,Kritter Planet,4.1,25,"Q2-Target,Optimized",1718.25,...,0.32,1230,298,354.0,458.82,14,0.0395,0.2670,211,Published
3,621903168,Catalonia Fleece Wearable Blanket Poncho for A...,1CTBP200PP,Bedding,Throws,Catalonia,0.0,0,"Hero 5 Holidays,WFS,WFS Item",1326.39,...,0.34,2926,365,2383.0,1055.49,50,0.0210,0.7958,0,Published
4,208255234,"Tirrinia Insulated Leakproof Lunch Bag, Adjust...",1TRLB701DGY-N,"Home Decor, Kitchen, & Other",Reusable Lunch Bags & Boxes,Tirrinia,4.6,169,"Q2-Target,Brand Camp VD,Lunch Bag,Optimized,WF...",1224.20,...,0.00,3037,406,2631.0,1224.20,101,0.0384,1.0000,247,Published
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,128458122,Deluxe Stylish Microplush Sleeved Fleece Blank...,k3000-pink,Clothing,Sleepwear Robes & Bathrobes,NAPA,0.0,0,"Fleece Blankets,Batch 1 - TR",-29.99,...,0.00,0,11,11.0,-29.99,-1,-0.0909,1.0000,0,Published
2375,788420613,Catalonia Sherpa Wearable Blanket Poncho for A...,1CTBP110WN,Clothing - Clothing,Throws,Catalonia,4.9,17,"Hero 5 Holidays,Batch 1 - TR",-32.99,...,0.23,0,31,23.0,-32.99,-1,-0.0435,1.0000,0,Published
2376,326279523,Catalonia 15lb Weighted Blanket for Kids Adult...,1CTBG724GY-15C,Bedding,Bed Blankets,Catalonia,4.2,9,"WFS,Valentine,WFS Item",-39.99,...,0.00,8,34,34.0,-39.99,-1,-0.0294,1.0000,0,Published
2377,113173878,Target Golf Laser Rangefinder 600M Flag Seeker...,TARGETGRF003WT,Sport & Recreation Other,Golf Rangefinders,Target,4.0,1,Optimized,-49.99,...,0.00,0,0,0.0,-49.99,-1,NaN,1.0000,0,Published


Sheet 'By Item IDs' imported as table 'ByItemIDs' with 2379 rows.
Processing summary table 'By Keywords' with standard headers


,01_01_2025___05_15_2025_135_days,Unnamed:_1,Unnamed:_2,Unnamed:_3,Unnamed:_4,Total_Ad_Revenue___Total_Adspend,Ad_Impressions_from_all_my_Ad_accounts,Unnamed:_7,Total_Ad_Clicks___Total_Ad_Impressions,Unnamed:_9,...,Unnamed:_12,Unnamed:_13,Unnamed:_14,Unnamed:_15,Unnamed:_16,Unnamed:_17,Unnamed:_18,Unnamed:_19,Items_that_rank_both_organically_and_through_Ads,Unnamed:_21
0,NaN,NaN,594.67,1420.46,1368.48,2.388653,67860,1483,0.021854,54,...,0.400991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Keyword,Search Volume(30 days),Ad Spend,Ad Revenue,Ad SKU Sales,ROAS,Ad Impressions,Ad Clicks,CTR,Ad Units Sold,...,CPC,Campaigns,Match Type,My Ranking Products,Best Organic Rank,My Sponsored Products,Best Sponsored Rank,Total Sponsored Products,Multiple Ranking Items,Ranking Items
2,picnic blanket,6470,37.62,34.99,34.99,0.93,13734,92,0.01,1,...,0.41,"1TRBW506GNBK_Manual,1TRBW506LGDG-CDCB_Manual,1...",exact,NaN,NaN,-,NaN,0,0,NaN
3,snuggie,620 - 7680,29.74,84.97,84.97,2.86,2494,74,0.03,3,...,0.4,"1CTBP120GY_Manual,1CTBP120GY_Manual_Optiwise,M...",exact,9,14,3,4,10,0,"873609379 - 32, 519184826 - 40, 768244496 - 14..."
4,stadium blanket,970,26.16,157.95,157.95,6.04,1275,76,0.06,5,...,0.34,"1CTBC200BKCAMO_Manual,1WMBW506NYGY-TBCB_Manual...",exact,9,5,3,1,10,1,"448077121 - 6, 888252893 - 5, 840040807 - 2, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,1228349737,NaN,0.3,0,0,0,2,1,0.5,0,...,0.3,Throws_Manual_Optiwise_24,phrase,NaN,NaN,-,NaN,0,0,NaN
476,1716270446,NaN,0.3,0,0,0,5,1,0.2,0,...,0.3,Throws_Manual_Optiwise_24,phrase,NaN,NaN,-,NaN,0,0,NaN
477,904345997,NaN,0.3,0,0,0,12,1,0.08,0,...,0.3,"1CTMM500PK_Manual_Optiwise,Throws_Manual_Optiw...",exact,NaN,NaN,0,NaN,0,0,NaN
478,15332523393,NaN,0.3,0,0,0,2,1,0.5,0,...,0.3,1CTBF579GNCAMO_Manual_Optiwise,exact,NaN,NaN,-,NaN,0,0,NaN


Sheet 'By Keywords' imported as table 'ByKeywords' with 480 rows.
Processed table 'Trend - Period' with combined headers


,Item_Id,Product_Name,SKU,Product_Type,Brand,col_5_Tags,GMV_Week_19_2025_05_05___2025_05_11,Units_Sold_Week_19_2025_05_05___2025_05_11,Page_Views_Week_19_2025_05_05___2025_05_11,Conversion_Rate_Week_19_2025_05_05___2025_05_11,...,Conversion_Rate_Week_16_2025_04_14___2025_04_20,Ad_Clicks_Week_16_2025_04_14___2025_04_20,Ad_Spend_Week_16_2025_04_14___2025_04_20,Ad_Sales_Week_16_2025_04_14___2025_04_20,ROAS_Week_16_2025_04_14___2025_04_20,CTR_Week_16_2025_04_14___2025_04_20,CPC_Week_16_2025_04_14___2025_04_20,TACOS_Week_16_2025_04_14___2025_04_20,AUR_Week_16_2025_04_14___2025_04_20,Impressions_Week_16_2025_04_14___2025_04_20
0,621903168,Catalonia Fleece Wearable Blanket Poncho for A...,1CTBP200PP,Throws,Catalonia,"Hero 5 Holidays,WFS,WFS Item",263.88,12,367,0.032698,...,0.013158,16,4.5,0,0,0.0105,0.28125,0.068213,21.99,1520
1,10286421989,Wrangler Men’s Leather Country Casual Every Da...,WR-BELT-MENS-COUNTRY-BLACK-40,Belts,Wrangler,Belts,159.92,8,6,1.333333,...,0,0,0,0,0,0,0,0,0,0
2,606072192,Single Wine Cooler Bags - Insulated & Padded P...,1WMWB101BG,"Wine Totes, Bags & Backpacks",Flyingstar,"Wine,Christmas,Optimized,Wine WFS,Valentine,SEM",143.88,12,4,3,...,0,0,0,0,0,0,0,0,0,0
3,775282043,"Tirrinia Insulated Lunch Bag for Women, Men, a...",1TRLB701CH-N,Reusable Lunch Bags & Boxes,Tirrinia,"Q2-Target,Brand Camp VD,Lunch Bag,WFS,Lunch Ba...",132.89,11,1065,0.010329,...,0.045963,0,0,0,0,0,0,0,12.678243,0
4,138121626,Realtree Unisex Hardwood Mesh Baseball Cap Out...,KC Caps HT01,Hats,KC Caps,"WFS,WFS Item",120,12,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,10305512889,Wrangler Men’s Leather Country Casual Every Da...,WR-BELT-MENS-COUNTRY-BLACK-42,Belts,Wrangler,Belts,-19.99,-1,0,0,...,0,0,0,0,0,0,0,0,19.99,0
2514,545745830,"Solaris Women's Large Wide Brim Hat, UV Protec...",1SLSH911-GN,Hats,Solaris,"Q2-Target,Optimized,WFS,Hats,WFS Item",-26.98,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
2515,9505771386,Wrangler Yellowstone Sturdy Laptop Backpack fo...,WR-BACKPACK-16273-BLKCORD,Laptop Bags,Wrangler,Backpack,-34.99,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
2516,15386919433,"Catalonia Swim Parka Kids with Hood, Lightweig...",1CTPKR202BK-0,Swim Parkas,Catalonia,NaN,-42.46,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


Sheet 'Trend - Period' imported as table 'TrendPeriod' with 2518 rows.
